TEAM -ID :PNT2022TMID15882
PROJECT NAME: AI-powered Nutrition Analyzer for Fitness Enthusiasts

In [23]:
pwd

'/home/wsuser/work'

In [24]:
!pip install keras
!pip install tensorflow

In [25]:
pip freeze

absl-py==0.12.0
aiohttp==3.8.1
aiosignal==1.2.0
appdirs==1.4.4
argon2-cffi==20.1.0
astropy==5.0
astunparse==1.6.3
async-generator==1.10
async-timeout==4.0.1
attrs==21.2.0
autoai-libs==1.13.6
autoai-ts-libs==1.1.9
autovizwidget==0.18.0
av==8.0.3
azure-core==1.21.0
azure-cosmos==4.2.0
backcall==0.2.0
beautifulsoup4==4.10.0
biopython==1.78
black==19.10b0
bleach==4.0.0
blinker==1.4
bokeh==2.3.3
boto3==1.18.21
botocore==1.21.41
Bottleneck==1.3.2
brotlipy==0.7.0
ca-data-connector @ file:///tmp/ws-build/install/ca-data-connector-python-11.2.4.tgz
cached-property==1.5.2
cachetools==4.2.2
cassandra-driver==3.25.0
certifi==2022.9.24
cffi==1.14.6
charset-normalizer==2.0.4
click==8.0.4
cloudpickle==1.6.0
cplex==20.1.0.2
cryptography==3.4.8
cx-Oracle==8.3.0
cycler==0.11.0
cytoolz==0.11.0
dask==2021.10.0
debugpy==1.5.1
decision-optimization-client==1.0.0
decorator==5.1.0
defusedxml==0.7.1
dill==0.3.4
dm-tree==0.1.5
docplex==2.22.213
entrypoints==0.3
et-xmlfile==1.1.0
flatbuffers==2.0
fonttools==4.25

In [26]:
import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.layers import Conv2D,MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

In [27]:
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

In [28]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='j-Jt-H2wxr3uqgpKqDSLrCgz2NtJzyF4VSIBD3iqSMjg',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'aipowerednutritionanalyzerforfitn-donotdelete-pr-xvq14lyx9l2gr1'
object_key = 'Dataset-Nutrition -Tracker.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [29]:
from io import BytesIO
import zipfile

unzip=zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths=unzip.namelist()

for path in file_paths:
    unzip.extract(path)

In [30]:
pwd

'/home/wsuser/work'

In [31]:
import glob

file_names=glob.glob("/home/wsuser/work/Dataset_Nutrition -Tracker/**")
file_names

['/home/wsuser/work/Dataset_Nutrition -Tracker/TRAIN_SET',
 '/home/wsuser/work/Dataset_Nutrition -Tracker/TEST_SET']

In [32]:
x_train = train_datagen.flow_from_directory(r'/home/wsuser/work/Dataset_Nutrition -Tracker/TRAIN_SET',
                                            target_size=(64, 64),
                                            batch_size=5,
                                            color_mode='rgb',
                                            class_mode='sparse')

x_test = test_datagen.flow_from_directory(r'/home/wsuser/work/Dataset_Nutrition -Tracker/TEST_SET',
                                          target_size=(64, 64),
                                          batch_size=5,
                                          color_mode='rgb',
                                          class_mode='sparse') 


Found 2626 images belonging to 5 classes.
Found 1055 images belonging to 5 classes.


In [33]:
print(x_train.class_indices)

{'APPLES': 0, 'BANANA': 1, 'ORANGE': 2, 'PINEAPPLE': 3, 'WATERMELON': 4}


In [34]:
from collections import Counter as ct
ct(x_train.labels)

Counter({0: 606, 1: 445, 2: 479, 3: 621, 4: 475})

In [35]:
classifier=Sequential()

In [36]:
classifier.add(Conv2D(32,(3,3),input_shape=(64,64,3),activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))

classifier.add(Conv2D(32,(3,3),activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))

classifier.add(Flatten())


In [37]:
classifier.add(Dense(units=128,activation='relu'))
classifier.add(Dense(units=5,activation='softmax'))

In [38]:
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 31, 31, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 6272)              0         
                                                                 
 dense_2 (Dense)             (None, 128)              

In [39]:
classifier.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [40]:

classifier.fit_generator(x_train,
                    steps_per_epoch = len(x_train) , 
                    epochs = 20, 
                    validation_data = x_test,
                    validation_steps = len(x_test) )

Epoch 1/20


/tmp/wsuser/ipykernel_164/4012943898.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(x_train,


526/526 [==============================] - 12s 23ms/step - loss: 0.1384 - accuracy: 0.9513 - val_loss: 0.0137 - val_accuracy: 1.0000
Epoch 2/20
526/526 [==============================] - 12s 22ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.0800 - val_accuracy: 0.9791
Epoch 3/20
526/526 [==============================] - 12s 22ms/step - loss: 1.2211e-04 - accuracy: 1.0000 - val_loss: 0.0527 - val_accuracy: 0.9791
Epoch 4/20
526/526 [==============================] - 12s 22ms/step - loss: 5.2475e-05 - accuracy: 1.0000 - val_loss: 0.0522 - val_accuracy: 0.9791
Epoch 5/20
526/526 [==============================] - 12s 23ms/step - loss: 2.1300e-05 - accuracy: 1.0000 - val_loss: 0.0550 - val_accuracy: 0.9791
Epoch 6/20
526/526 [==============================] - 12s 23ms/step - loss: 1.2199e-05 - accuracy: 1.0000 - val_loss: 0.0656 - val_accuracy: 0.9791
Epoch 7/20
526/526 [==============================] - 12s 22ms/step - loss: 1.1297e-05 - accuracy: 1.0000 - val_loss: 0.0612 - val_

In [19]:
classifier.save('Nutrition-Analysis.h5')


In [20]:
ls

'Dataset_Nutrition -Tracker'/   Nutrition-Analysis.h5


In [21]:
!tar -zcvf nutrition-tracker-trained-model.tgz Nutrition-Analysis.h5

Nutrition-Analysis.h5


In [22]:
ls

'Dataset_Nutrition -Tracker'/   nutrition-tracker-trained-model.tgz
 Nutrition-Analysis.h5


In [24]:
!pip install watson-machine-learning-client --upgrade

In [28]:
from ibm_watson_machine_learning import APIClient

wml_credentials={
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"X7wa8AOyA3srcfISiRjUOT81xGtbBdV-ogBI-GstTxJA"
}

In [29]:
client=APIClient(wml_credentials)
client

In [30]:
def guid_from_space_name(client,space_name):
    space=client.spaces.get_details()
#     print(space)
    return(next(item for item in space["resources"] if item['entity']['name']==space_name)['metadata']['id'])

In [31]:
space_uid=guid_from_space_name(client,"Nutrition_Analyzer_For_Fitness_Enthusiasts")
print("space_uid ="+space_uid)

space_uid =3cd4fe5b-49f1-4ae7-bdb8-d419fe1e7482


In [32]:
client.set.default_space(space_uid)

'SUCCESS'

In [33]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [34]:
software_spec_uid=client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid


'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [35]:
model_details=client.repository.store_model(model='nutrition-tracker-trained-model.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN Model Building",
    client.repository.ModelMetaNames.TYPE:'tensorflow_2.7',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid
})
model_id=client.repository.get_model_uid(model_details)

This method is deprecated, please use get_model_id()


/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/repository.py:1453: UserWarning: This method is deprecated, please use get_model_id()
  warn("This method is deprecated, please use get_model_id()")


In [36]:
model_id

'8d837d30-99f1-40e5-bb30-2efd72596047'

In [37]:
client.repository.download(model_id,"my_model.tar.gz")

Successfully saved model content to file: 'my_model.tar.gz'


'/home/wsuser/work/my_model.tar.gz'

In [45]:
ls

'Dataset_Nutrition -Tracker'/   Nutrition-Analysis.h5
 my_model.tar.gz                nutrition-tracker-trained-model.tgz


In [38]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image
import keras
import numpy as np
import glob
import matplotlib.pyplot as plt

In [39]:
model=load_model("Nutrition-Analysis.h5")

In [44]:

# # import os, types
# # import pandas as pd
# # from botocore.client import Config
# # import ibm_boto3

# # def __iter__(self): return 0

# # # @hidden_cell
# # # The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# # # You might want to remove those credentials before you share the notebook.
# # cos_client = ibm_boto3.client(service_name='s3',
# #     ibm_api_key_id='j-Jt-H2wxr3uqgpKqDSLrCgz2NtJzyF4VSIBD3iqSMjg',
# #     ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
# #     config=Config(signature_version='oauth'),
# #     endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

# bucket = 'aipowerednutritionanalyzerforfitn-donotdelete-pr-xvq14lyx9l2gr1'
# object_key = 'test_1.jpg'

# streaming_body_3 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# # Your data file was loaded into a botocore.response.StreamingBody object.
# # Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# # ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# # pandas documentation: http://pandas.pydata.org/
# img=image.load_img(streaming_body_2,target_size=(64,64))


TypeError: expected str, bytes or os.PathLike object, not StreamingBody

In [41]:


x=keras.utils.img_to_array(img)
x=np.expand_dims(x,axis=0)

pred=model.predict(x)
pred

TypeError: expected str, bytes or os.PathLike object, not StreamingBody

In [ ]:
np.argmax(pred)

In [ ]:
index=["APPLES","BANANAS","ORANGE","PINEAPPLE","WATERMELON"]
result=str(index[np.argmax(pred)])
result

In [ ]:
fig=plt.figure(figsize=(5,5))
count=0
testing_imgs=glob.glob("*.jpg")
for i in testing_imgs:
    count+=1
    img=keras.utils.load_img(i,grayscale=False,target_size=(64,64))
    

    x=keras.utils.img_to_array(img)
    x=np.expand_dims(x,axis=0)

    pred=model.predict(x)
    print(pred)
    # plt.imshow(img)
    result=str(index[np.argmax(pred)])
    print(result)
    fig=plt.figure(figsize=(5,5))
    plt.title(result)
    plt.axis("off")
    plt.imshow(img)